In [1]:
# %matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from census import Census
import requests
from datetime import datetime


# Census API key
from config import api_key

# Access raw housing data

metro_hv_df = pd.read_csv("Data/Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv")
metro_inventory_df = pd.read_csv("Data/Metro_invt_fs_uc_sfr_sm_month.csv")
zipcode_hv_df = pd.read_csv("Data/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv")
census_df = pd.read_csv("Data/Census/all_census_data_zipcode.csv")

# Filter all the dataframes to select only Atlanta Metro

metro_hv_df = metro_hv_df.loc[metro_hv_df["RegionName"] == "Atlanta, GA"]
metro_inventory_df = metro_inventory_df.loc[metro_inventory_df["RegionName"] == "Atlanta, GA"]
zipcode_hv_df = zipcode_hv_df.loc[zipcode_hv_df["Metro"] == "Atlanta-Sandy Springs-Roswell"]

In [2]:
zipcode_hv_df = zipcode_hv_df.set_index("RegionName")
zipcode_hv_df = zipcode_hv_df.iloc[:, 8:]

zipcode_hv_df

,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,...,9/30/2020,10/31/2020,11/30/2020,12/31/2020,1/31/2021,2/28/2021,3/31/2021,4/30/2021,5/31/2021,6/30/2021
RegionName,,,,,,,,,,,,,,,,,,,,,
30349,73512.0,74617.0,75789.0,76978.0,78192.0,79264.0,80501.0,81908.0,83782.0,86172.0,...,173192,175092,177039,179313,181593,184321,187589,191315,195593,200055
30044,106773.0,109451.0,112004.0,114353.0,116818.0,119171.0,121781.0,124753.0,128241.0,131163.0,...,232282,235282,238202,241419,245063,249666,254447,259789,266046,273778
30043,150801.0,153675.0,155980.0,158203.0,160981.0,163558.0,165971.0,168538.0,171970.0,175002.0,...,269260,272660,276000,279810,284199,289620,295337,301752,309502,318962
30096,141225.0,143945.0,146488.0,149209.0,151740.0,154078.0,156148.0,159071.0,162738.0,166312.0,...,267723,270830,273698,277059,281030,286211,291781,297841,305055,313602
30024,250908.0,254510.0,257754.0,260711.0,263957.0,266781.0,269089.0,271866.0,275937.0,279850.0,...,385998,389679,393932,398708,404112,410824,418020,426334,436153,448204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30177,93552.0,94348.0,94870.0,95427.0,95944.0,96490.0,96905.0,97466.0,98104.0,98675.0,...,150684,150827,151907,153187,154883,155815,156790,157333,158274,159285
30111,78863.0,80089.0,81748.0,82909.0,83422.0,83279.0,83798.0,85351.0,86995.0,86388.0,...,158907,160941,163740,166334,168236,171570,175838,182894,190439,197758
30623,127038.0,127056.0,127065.0,127412.0,128189.0,128742.0,129569.0,130274.0,130906.0,131183.0,...,175878,176562,178679,180283,182176,183376,184530,185790,187208,189575


In [4]:
zipcode_hv_df.transpose()

RegionName,30349,30044,30043,30096,30024,30281,30040,30318,30188,30052,...,30137,30148,30250,31085,31038,30177,30111,30623,30151,30289
1/31/2013,73512.0,106773.0,150801.0,141225.0,250908.0,96004.0,241044.0,117056.0,176730.0,131831.0,...,95438.0,132522.0,NaN,100692.0,78978.0,93552.0,78863.0,127038.0,38294.0,174773.0
2/28/2013,74617.0,109451.0,153675.0,143945.0,254510.0,98173.0,243258.0,117423.0,178180.0,134647.0,...,96910.0,133283.0,NaN,100902.0,79305.0,94348.0,80089.0,127056.0,38381.0,175934.0
3/31/2013,75789.0,112004.0,155980.0,146488.0,257754.0,100477.0,245242.0,118480.0,179777.0,136862.0,...,98301.0,133892.0,NaN,101370.0,79385.0,94870.0,81748.0,127065.0,38199.0,176836.0
4/30/2013,76978.0,114353.0,158203.0,149209.0,260711.0,102948.0,248100.0,119592.0,181643.0,139053.0,...,98842.0,134643.0,NaN,101767.0,79694.0,95427.0,82909.0,127412.0,37917.0,177681.0
5/31/2013,78192.0,116818.0,160981.0,151740.0,263957.0,104422.0,250398.0,121700.0,183702.0,140940.0,...,99455.0,135408.0,NaN,102282.0,80051.0,95944.0,83422.0,128189.0,37723.0,179185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2/28/2021,184321.0,249666.0,289620.0,286211.0,410824.0,207780.0,390755.0,302692.0,321932.0,266045.0,...,189968.0,216999.0,174530.0,164722.0,125189.0,155815.0,171570.0,183376.0,64046.0,296811.0
3/31/2021,187589.0,254447.0,295337.0,291781.0,418020.0,211728.0,398066.0,307130.0,326864.0,271094.0,...,192495.0,218303.0,175684.0,165988.0,125716.0,156790.0,175838.0,184530.0,64444.0,301913.0
4/30/2021,191315.0,259789.0,301752.0,297841.0,426334.0,215814.0,406892.0,312327.0,332191.0,276610.0,...,195523.0,219197.0,178456.0,167157.0,126444.0,157333.0,182894.0,185790.0,65498.0,308534.0
5/31/2021,195593.0,266046.0,309502.0,305055.0,436153.0,221143.0,416541.0,318866.0,338264.0,282841.0,...,199434.0,220945.0,185625.0,168846.0,127391.0,158274.0,190439.0,187208.0,67434.0,314300.0
